Задание 1

In [0]:
import pandas as pd


In [2]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv
Saving weatherHistory.csv to weatherHistory.csv


In [0]:
weather_hist = pd.read_csv('weatherHistory.csv')

Меняем названия, удаляем ненужный столбец, переводим даты в удобный формат

In [0]:
weather_hist.rename({'Formatted Date':'date', 'Temperature (C)':'temp', 'Apparent Temperature (C)':'temp_app',
                    'Precip Type':'weather_type', 'Wind Speed (km/h)':'wind_speed', 'Humidity':'humidity',
                    'Daily Summary':'summary'}, 
                    axis=1, inplace=True)
weather_hist.drop(['Loud Cover'], axis=1, inplace=True)
weather_hist['date'] = pd.to_datetime(weather_hist['date'], utc = True)

Найдем самый холодный и самый ветренный дни. Будем считать самым холодным тот день, когда была зафиксирована самая холодная температура, хотя для большей точности следовало бы считать среднюю температуру по каждому из дней. Аналогично с ветром.

In [0]:
the_coldest_day = weather_hist[weather_hist['temp'] == weather_hist['temp'].min()]
the_coldest_day.head()

,date,Summary,weather_type,temp,temp_app,humidity,wind_speed,Wind Bearing (degrees),Visibility (km),Pressure (millibars),summary
54847,2012-02-10 06:00:00+00:00,Foggy,snow,-21.822222,-21.822222,0.8,3.0751,323.0,1.3685,1033.66,Foggy starting in the morning continuing until...


In [0]:
the_windiest_day = weather_hist[weather_hist['wind_speed'] == weather_hist['wind_speed'].max()]
the_windiest_day.head()

,date,Summary,weather_type,temp,temp_app,humidity,wind_speed,Wind Bearing (degrees),Visibility (km),Pressure (millibars),summary
12181,2007-01-29 12:00:00+00:00,Dangerously Windy and Partly Cloudy,rain,8.944444,3.483333,0.49,63.8526,307.0,11.4471,1009.05,Mostly cloudy throughout the day and windy sta...


Добавим колонки с годами, месяцами и неделями

In [0]:
weather_hist['year'] = weather_hist['date'].dt.year
weather_hist['month'] = weather_hist['date'].dt.month
weather_hist['week'] = weather_hist['date'].dt.week

Создаем таблицу с дождями за лето 2014 года

In [0]:
rain_2014 = weather_hist[(weather_hist['weather_type'] == 'rain') & (weather_hist['year'] == 2014) & (weather_hist['month'].isin(['6','7','8']))]

In [0]:
print('количество зафиксированных дождей летом 2014 года -', len(rain_2014))

количество зафиксированных дождей летом 2014 года - 2205


In [0]:
weather_hist['year'].value_counts() #проверим сколько имеется данных по каждому году

2012    8784
2010    8784
2008    8784
2016    8783
2015    8760
2013    8760
2011    8760
2009    8760
2007    8760
2006    8760
2014    8757
2005       1
Name: year, dtype: int64

В 2005 году представлена только одна строка, что не очень репрезентативно по сравнению с другими годами, поэтому удалим её

In [0]:
weather_hist = weather_hist.loc[weather_hist['year'] != 2005] 



Самый влажный год

In [0]:
years = {}
for year in set(weather_hist['year']):
  years[year] = weather_hist[weather_hist['year'] == year]

In [0]:
anual_humidity = {}
for year in years:
  anual_humidity[year] = years[year]['humidity'].mean()

In [0]:
max_value = max(anual_humidity.values())
for year in anual_humidity:
  if anual_humidity[year]==max_value:
    print('самый влажный год -', year, 
          'со средним уровнем влажности -', max_value) 

самый влажный год - 2010 со средним уровнем влажности - 0.7968613387978182


In [0]:
months = {}
for year in set(weather_hist['year']):
  for month in set(weather_hist['month']):
    months[str(month)+str(year)] = weather_hist[(weather_hist['year'] == year) & (weather_hist['month'] == month)]

In [0]:
wind_per_month = {}
for month in months:
  wind_per_month[month] = months[month]['wind_speed'].mean()

In [0]:
max_wind = max(wind_per_month.values())
for month in months:
  if wind_per_month[month]==max_wind:
    print('самый ветренный месяц -', month[:-4]+'.'+month[-4:], 
          'со средней скоростью ветра -', max_wind) 

самый ветренный месяц - 3.2009 со средней скоростью ветра - 16.88236478494623


"Самая переменчивая погода" - неоднозначная формулировка. Будем учитывать те величины, по которым всегда есть информация: скорость ветра и температура.

In [0]:
weeks = {}
for year in years:
  for week in set(weather_hist['week']):
    weeks[str(week)+str(year)] = weather_hist[(weather_hist['year'] == year) & (weather_hist['week'] == week)]

In [0]:
the_most_changeable = {} # будем заносить разницу между максимумом и минимумом по температуре и ветру для каждой недели
temp, wind_speed= {},  {}
for week in weeks:
  temp[week] = weeks[week]['temp'].max() - weeks[week]['temp'].min()
  wind_speed[week] = weeks[week]['wind_speed'].max() - weeks[week]['wind_speed'].min()

In [0]:
ampl_ws = max(wind_speed.values())
ampl_t = max(temp.values())
for week in weeks:
  if wind_speed[week]==ampl_ws:
    print('неделя с самым переменчивым ветром -', week[:-4]+'.'+week[-4:],
    'амплитуда скорости ветра -', ampl_ws)
  if temp[week]==ampl_t:
    print('неделя с самой переменчивой t -', week[:-4]+'.'+week[-4:],
    'амплитуда температуры -', ampl_t)

неделя с самым переменчивым ветром - 5.2007 амплитуда скорости ветра - 61.2444
неделя с самой переменчивой t - 52.2009 амплитуда температуры - 32.76111111111111


In [0]:
week_52_2009 = weather_hist[(weather_hist['week'] == 52) & (weather_hist['year'] == 2009)]
week_52_2009.groupby('temp').agg(['mean']) # проверяем разброс температуры 

In [0]:
week_05_2007 = weather_hist[(weather_hist['week'] == 5) & (weather_hist['year'] == 2007)]
week_05_2007.groupby('wind_speed').agg(['mean']) # проверяем разброс ветра

Задание 2

In [0]:
data = pd.read_csv('data.csv')

In [0]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import LinearSVR
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Уже имеющиеся признаки.

In [0]:
data['text_len'] = data['description'].apply(len)
data['text_len_tokens'] = data['description'].apply(lambda x: len(x.split()))
data['number_of_caps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isupper()]))
data['number_of_nonalphanum'] = data['description'].apply(lambda x: len([ch for ch in x if not ch.isalnum()]))

Новые признаки. Сначала напишем функции с помощью регулярок.

In [0]:
import re

#наличие цифр, т.к. они обычно присутствуют в подробном описании
def digits (text):
  d = re.findall('[1-9]', text)
  return len(d)

#доля латинских символов, т.к. приведенные в оригинале названия изделий внушают больше доверия
def latin(text):
  l = re.findall('[a-z][A-Z]', text)
  return len(l)/len(text)

#доля кириллических символов, т.к. высокая доля текста делают описание "живее"
def cyrillic(text):
  c = re.findall('[а-я][А-Я]', text)
  return len(c)/len(text)

#наличие "продающих слов" в тексте, т.к. возможно они могут повлиять на решение человека о покупке
def magic_lemmas(text):
  magic_lemmas = ['идеальн','отличн', 'прекрасн', 'хорош', 'нов', 'качеств', 'свеж', 'великолепн', 'лучш']
  total_len = 0
  for lemma in magic_lemmas:
    magic = re.findall(lemma, text)
    total_len += len(magic)
  return total_len

#указана причина продажи, т.к. внушает больше доверия, покупатель понимает, что ему не пытаются "сбагрить" сломавшуюся вещь
def why(text):
  why = 'причина продажи'
  why_finder = re.findall(why, text.lower())
  return len(why_finder)


#есть слово торг (предположительно наличие этого слова означает возможность торга), т.к. демонстрирует открытость к диалогу и можно уменьшить цену
def torg(text):
  market = re.findall('торг', text.lower())
  return len(market)  


# последние 3 признака возможно не очень универсальны, не во всех текстах встречаются такие слова

In [0]:
data['digits'] = data['description'].apply(digits)
data['latin'] = data['description'].apply(latin)
data['cyrillic'] = data['description'].apply(cyrillic)
data['magic_lemmas'] = data['description'].apply(magic_lemmas)
data['why'] = data['description'].apply(why)
data['torg'] = data['description'].apply(torg)

In [0]:
X = data[['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'digits', 'latin', 'cyrillic', 'magic_lemmas', 'why', 'torg']].values
y = data.deal_probability.values

In [92]:
X.shape

(138715, 10)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [93]:
regressor = Ridge(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))


0.26304963770431733

Всё стало немного хуже

In [68]:
data.corr() # посмотрим на корреляцию

,price,item_seq_number,image_top_1,deal_probability,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,digits,latin,cyrillic,magic_lemmas,why,torg
price,1.000000,0.077973,0.046540,-0.015032,0.070124,0.066481,0.017722,0.056030,0.021605,-0.008075,0.000909,0.031499,-0.004599,0.035284
item_seq_number,0.077973,1.000000,0.093411,-0.038196,0.117144,0.112789,0.079725,0.127046,0.057301,-0.005108,0.060832,0.057765,-0.007406,0.012111
image_top_1,0.046540,0.093411,1.000000,0.185397,0.184829,0.182593,0.150551,0.174994,0.123618,0.107693,0.039923,0.046216,0.022933,0.092761
deal_probability,-0.015032,-0.038196,0.185397,1.000000,-0.009304,-0.013111,-0.012485,-0.023144,-0.028328,0.014381,-0.009749,0.005525,0.007982,0.041524
text_len,0.070124,0.117144,0.184829,-0.009304,1.000000,0.989847,0.612836,0.944754,0.486967,0.024564,0.050950,0.484854,0.025061,0.080893
text_len_tokens,0.066481,0.112789,0.182593,-0.013111,0.989847,1.000000,0.618434,0.938101,0.501690,0.028282,0.049508,0.472494,0.027134,0.078379
number_of_caps,0.017722,0.079725,0.150551,-0.012485,0.612836,0.618434,1.000000,0.628858,0.464501,0.049629,0.057320,0.199391,0.009786,0.039602
number_of_nonalphanum,0.056030,0.127046,0.174994,-0.023144,0.944754,0.938101,0.628858,1.000000,0.520895,0.029485,0.059073,0.414269,0.023465,0.067568
digits,0.021605,0.057301,0.123618,-0.028328,0.486967,0.501690,0.464501,0.520895,1.000000,0.053331,0.068968,0.135133,0.007747,0.012640
latin,-0.008075,-0.005108,0.107693,0.014381,0.024564,0.028282,0.049629,0.029485,0.053331,1.000000,0.007198,-0.000034,0.008655,-0.005810


Самый высокий уровень корреляции у наличия слова "торг"! Высокий уровень корреляции(со знаком минус) с вероятностью сделки у цифровых символов(превосходит даже предложенные раньше факторы). Указанная причина продажи коррелирует с продажей больше, чем "продающие слова". Высокая доля латинских символов тоже играет важную роль. Доля кириллических символов оказывается важнее, чем продающие слова и указанная причина продажи. 

In [88]:
X = data[['torg']].values
y = data.deal_probability.values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
regressor = Ridge(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2634532789804883

Если предсказывать только по слову "торг", то результаты незначительно улучшаются (0.2634532789804883). Увеличение параметров обычно ухудшает результат (возможно, надо подобрать правильную комбинацию). Предсказание по продающим словам оказывается еще немного лучше (возможно текстов, содержащих такие слова всё-таки больше) - 0.26366562582516456. Предсказание по причине продажи похоже - 0.26366049220093507. Очень смущает близость таких предсказаний. 

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [0]:
def prediction (cv):
  X = cv.fit_transform(data.description)
  y = data.deal_probability.values
  X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
  regressor = Ridge(alpha=10)
  regressor.fit(X_train, y_train)
  preds = regressor.predict(X_test)
  return np.sqrt(mean_squared_error(y_test, preds))

Изучим параметры векторизатора. В учебной тетради они были следующими:

In [118]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2))
prediction(cv)

0.2512515450632489

Попробуем уменьшить параметр max_features

In [184]:
cv = CountVectorizer(max_features=1, min_df=5, max_df=0.4, ngram_range=(1,2))
prediction(cv)

0.26357648533064765

Результаты стали лучше

Попробуем увеличить параметр min_df

In [160]:
cv = CountVectorizer(max_features=1000, min_df=10000, max_df=0.4, ngram_range=(1,2))
prediction(cv)

0.2616050557339977

Изменение на 1%

Изменим max_df

In [121]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.0001, ngram_range=(1,2))
prediction(cv)

0.2637544732105779

При удалении слов, которые появляются более, чем в 10% текстов (вместо 40), результаты немного улучшаются - 0.25156431949139557 вместо 0.2512515450632489. Для 0.01% документов - 0.2637544732105779 (выше более чем на 1%!). Сделать данный параметр меньше не получается.

Попробуем учитывать триграммы

In [122]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,3))
prediction(cv)

0.2512626583953009

Незначительно улучшились результаты c триграммами

Попробуем изменить параметры max_features, min_df и max_df вместе, чтобы достичь максимального результата






In [171]:
cv = CountVectorizer(max_features=1000, min_df=1, max_df=0.0001, ngram_range=(1,2))
prediction(cv)

0.26410493207025837

Попробуем сделать то же самое с Tfidf. Сначала было так:

In [123]:
cv = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.4)
prediction(cv)

0.248098946453737

In [182]:
cv = TfidfVectorizer(max_features=1, min_df=5, max_df=0.4)
prediction(cv)

0.26360133315139683

In [125]:
cv = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.0001)
prediction(cv)

0.263257528400969

Уменьшение max_df до 0.0001 снова показывает улучшение результатов

In [153]:
cv = TfidfVectorizer(max_features=1000, min_df=10000, max_df=0.4)
prediction(cv)

0.26163943349680596

Значительное увеличение min_df улучшает результаты

Попробуем изменить параметры max_features, min_df и max_df вместе, чтобы достичь максимального результата

In [181]:
cv = TfidfVectorizer(max_features=1, min_df=1, max_df=0.0001)
prediction(cv)

0.26367373768000607